# Grid Layout Visualization

This code is meant to run inside the docker container. If you're running it locally, please:
1. Substitute the files **ModularVisualization.py** and **ChartModule.js** by the ones in the _lib_ subdirectory provided (the original versions are also in the directory if needed);
2. Uncomment and run the following cell:

In [1]:
# import sys
# sys.path.insert(0, '../..')

The code in this notebook requires a few changes in the code of the imported libraries, namely:
- ModularVisualization.py -> class ModularServer -> \_\_init\_\_() : to allow the server to run the already instantiated object of the CovidModel class (instead of instantiating it inside the server). **WARNING: This also disables the "Reset" button once the simulation is running.**
- ChartModule.js -> var ChartModule -> chartOptions: to improve visualizations by adding axes labels and fixing the manually defined maximum value of 900 to the chart. 

**To do that, please use the Dockerfile and "run-container.sh" script provided in this branch.** It will also map a new port into the container so that you can access the simulation interface at ( http://127.0.0.1:8521 ).

In [2]:
import copy
import numpy as np
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci, get_parameters
from utils import BasicStatistics, RemovePolicy, Propaganda, setup_grid_layout, setup_city_layout, AddPolicy, AddPolicyInfectedRate, confidence_interval#, multiple_runs
from model.utils import SocialPolicy
from model.human import Elder, Adult, K12Student, Toddler, Infant, Human
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.ModularVisualization import ModularServer


In [3]:
def compute_color_intensity(color, v, lower_bound, upper_bound):
    """
    Computes color intensity - alpha - value from lower_bound (minimum) to upper_bound(maximum), proportionally. Returns the modified HTML color (and intensity) string.
    
    :param color: HTML color string;
    :param v: current value;
    :param lower_bound: maximum intensity value;
    :param upper_bound: minimum intensity value;
    :return: HTML color string for the selected color with intensity (alpha) varying from 0 to 100% based on the value range. 
    """
    if v > upper_bound:
        print("Warning: alpha value greater than upper bound!!!")
        v = upper_bound
    elif v < lower_bound:
        print("Warning: alpha value smaller than lower bound!!!")
        v = lower_bound
    max_alpha = 255
    min_alpha = 0
    v = (((v - lower_bound) / (upper_bound - lower_bound)) * (max_alpha - min_alpha)) + min_alpha
    if v > 255: v = 255
    alpha = format(int(round(v)), '02x').upper()
    return color + alpha


def agent_portrayal(agent):
    """
    Agent portrayal method for MESA component. Defines how each agent on the grid will be displayed.
    
    :param agent: agent in the simulation grid;
    :return: an agent portrayal dictionary.
    """
    portrayal = {"Shape": "rect",
                 "Filled": "true"}
    if agent.name.startswith("Home"):
        humans = [h for h in agent.allocation if isinstance(h, Human)]
        num_humans = len(humans)
        num_infected = len([h for h in humans if h.is_infected()])
#         all_humans = [h for h in agent.covid_model.agents if isinstance(h, Human)]
#         humans = [h for h in all_humans if h.home_district and h.home_district.strid == agent.strid]
#         num_humans = len(humans)
#         num_infected = len([h for h in humans if h.is_infected()])
        num_hospitalized = len([h for h in humans if h.is_hospitalized()])
        if num_hospitalized > num_humans * get_parameters().params['hospitalization_capacity']:
            portrayal = {"Shape": "rect",
                        "Filled": "true",
                        "Color": "#FC9803FF",
                        "w": .7,
                        "h": .7,
                        "Layer": 1}
            return portrayal
        portrayal["w"] = .7
        portrayal["h"] = .7
        portrayal["Layer"] = 1
        if num_infected >= 1:
            portrayal["Color"] = compute_color_intensity('#FF0A0A', num_infected, 0, 7)
        else:
            portrayal["Color"] = "#0AFF0E55"
        return portrayal
    elif agent.name.startswith("F_"):
        real_agent_name = agent.name.split('_')[1]
        all_humans = [h for h in agent.covid_model.agents if isinstance(h, Human)]
        humans = [h for h in all_humans if h.home_district and h.home_district.name == real_agent_name]
        num_dead = len([h for h in humans if h.is_dead])
        if num_dead >= 1:
            portrayal = {"Shape": "circle",
                        "Filled": "true",
                        "Color": "#000000AA",
                        "r": num_dead/10,
                        "Layer": 2}
            return portrayal
        else:
            return None
    elif agent.name.startswith("School"):
        humans = [h for h in agent.allocation if isinstance(h, Human)]
        num_humans = len(humans)
        num_infected = len([h for h in humans if h.is_infected()])
#         all_humans = [h for h in agent.covid_model.agents if isinstance(h, Human)]
#         humans = [h for h in all_humans if h.school_district and h.school_district.strid == agent.strid]
#         num_humans = len(humans)
#         num_infected = len([h for h in humans if h.is_infected()])
#         print(agent.name + " Humans: " + str(num_humans) + " Infected: " + str(num_infected))
        portrayal["w"] = 1.
        portrayal["h"] = 1.
        portrayal["Layer"] = 0 
        print("SCHOOL infected proportion: " + str(np.round(num_infected / num_humans,2)))
        if (num_infected / num_humans) >= 0.02:
            portrayal["Color"] = "#FC7B0355"
        else:
            portrayal["Color"] = "#00FF9155"
        return portrayal
    return portrayal


class InfectedText(TextElement):
    """
    Displays a centralized, padded version of the input text (defined on the constructor) on MESA server.
    """
    
    def __init__(self, text):
        self.text = "<p style=\"text-align:center;padding-top:20px;\">" + text + "</p>"
        pass
    def render(self, model):
        return self.text

## Scenarios
The following cell contains parameters for 6 different scenarios following Ben's suggestions.

In [4]:
# Fixed Parameters
population_size = 1000
simulation_cycles = 60 # days
seed = 5

# Changing parameters
scenarios_dict = {
    # 1) No clumpiness, no lockdown (lots of spreading)
    'scenario_1': { 'parameters': SimulationParameters(
                                    mask_user_rate = 0.0,
                                    mask_efficacy = 0.0,
                                    imune_rate = 0.01,
                                    initial_infection_rate = 0.01,
                                    hospitalization_capacity = 0.05,
                                    latency_period_shape = 3,
                                    latency_period_scale = 1, 
                                    incubation_period_shape = 6,
                                    incubation_period_scale = 1, 
                                    mild_period_duration_shape = 14,
                                    mild_period_duration_scale = 1,
                                    hospitalization_period_duration_shape = 12,
                                    hospitalization_period_duration_scale = 1,
                                    symptomatic_isolation_rate = 0.0,
                                    asymptomatic_contagion_probability = 0.1,
                                    risk_tolerance_mean = 0.7,
                                    risk_tolerance_stdev = 0.2,
                                    herding_behavior_mean = 0.7,
                                    herding_behavior_stdev = 0.2,
                                    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
                                    spreading_rate = normal_ci(2.41, 3.90, 20)
                                ),
                   'listeners': [],
                        'home_grid_height': 6,
                        'home_grid_width' : 6,
                        'school_height' : 2,
                        'school_width' : 2,
                        'work_height' : 3,
                        'work_width' : 3
                  },
    # 2) High clumpiness, short lockdown (not such terrible spreading)
    'scenario_2': { 'parameters': SimulationParameters(
                                    mask_user_rate = 0.0,
                                    mask_efficacy = 0.0,
                                    imune_rate = 0.01,
                                    initial_infection_rate = 0.01,
                                    hospitalization_capacity = 0.05,
                                    latency_period_shape = 3,
                                    latency_period_scale = 1, 
                                    incubation_period_shape = 6,
                                    incubation_period_scale = 1, 
                                    mild_period_duration_shape = 14,
                                    mild_period_duration_scale = 1,
                                    hospitalization_period_duration_shape = 12,
                                    hospitalization_period_duration_scale = 1,
                                    symptomatic_isolation_rate = 0.0,
                                    asymptomatic_contagion_probability = 0.1,
                                    risk_tolerance_mean = 0.7,
                                    risk_tolerance_stdev = 0.2,
                                    herding_behavior_mean = 0.7,
                                    herding_behavior_stdev = 0.2,
                                    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
                                    spreading_rate = normal_ci(2.41, 3.90, 20)
                                ),
                   'listeners': [
                            ["AddPolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_RETAIL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_FACTORY, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_OFFICE, 20],
                        ],
                        'home_grid_height': 6,
                        'home_grid_width' : 6,
                        'school_height' : 2,
                        'school_width' : 1,
                        'work_height' : 1,
                        'work_width' : 2
                  },
    # 3) Low clumpiness, short lockdown (lots of spreading). 
    'scenario_3': { 'parameters': SimulationParameters(
                                    mask_user_rate = 0.0,
                                    mask_efficacy = 0.0,
                                    imune_rate = 0.01,
                                    initial_infection_rate = 0.01,
                                    hospitalization_capacity = 0.05,
                                    latency_period_shape = 3,
                                    latency_period_scale = 1, 
                                    incubation_period_shape = 6,
                                    incubation_period_scale = 1, 
                                    mild_period_duration_shape = 14,
                                    mild_period_duration_scale = 1,
                                    hospitalization_period_duration_shape = 12,
                                    hospitalization_period_duration_scale = 1,
                                    symptomatic_isolation_rate = 0.0,
                                    asymptomatic_contagion_probability = 0.1,
                                    risk_tolerance_mean = 0.7,
                                    risk_tolerance_stdev = 0.2,
                                    herding_behavior_mean = 0.7,
                                    herding_behavior_stdev = 0.2,
                                    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
                                    spreading_rate = normal_ci(2.41, 3.90, 20)
                                ),
                   'listeners': [
                            ["AddPolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_RETAIL, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_FACTORY, 20],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_OFFICE, 20],
                        ],
                        'home_grid_height': 6,
                        'home_grid_width' : 6,
                        'school_height' : 2,
                        'school_width' : 2,
                        'work_height' : 3,
                        'work_width' : 3
                  },
    # 4) High clumpiness, long lockdown (also not such terrible spreading). 
    'scenario_4': { 'parameters': SimulationParameters(
                                    mask_user_rate = 0.0,
                                    mask_efficacy = 0.0,
                                    imune_rate = 0.01,
                                    initial_infection_rate = 0.01,
                                    hospitalization_capacity = 0.05,
                                    latency_period_shape = 3,
                                    latency_period_scale = 1, 
                                    incubation_period_shape = 6,
                                    incubation_period_scale = 1, 
                                    mild_period_duration_shape = 14,
                                    mild_period_duration_scale = 1,
                                    hospitalization_period_duration_shape = 12,
                                    hospitalization_period_duration_scale = 1,
                                    symptomatic_isolation_rate = 0.0,
                                    asymptomatic_contagion_probability = 0.1,
                                    risk_tolerance_mean = 0.7,
                                    risk_tolerance_stdev = 0.2,
                                    herding_behavior_mean = 0.7,
                                    herding_behavior_stdev = 0.2,
                                    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
                                    spreading_rate = normal_ci(2.41, 3.90, 20)
                                ),
                   'listeners': [
                            ["AddPolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_RETAIL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_FACTORY, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_OFFICE, 1],
                        ],
                        'home_grid_height': 6,
                        'home_grid_width' : 6,
                        'school_height' : 2,
                        'school_width' : 1,
                        'work_height' : 1,
                        'work_width' : 2
                  },
    # 5) No clumpiness, severe lockdown (very little spreading).
    'scenario_5': { 'parameters': SimulationParameters(
                                    mask_user_rate = 0.0,
                                    mask_efficacy = 0.0,
                                    imune_rate = 0.01,
                                    initial_infection_rate = 0.01,
                                    hospitalization_capacity = 0.05,
                                    latency_period_shape = 3,
                                    latency_period_scale = 1, 
                                    incubation_period_shape = 6,
                                    incubation_period_scale = 1, 
                                    mild_period_duration_shape = 14,
                                    mild_period_duration_scale = 1,
                                    hospitalization_period_duration_shape = 12,
                                    hospitalization_period_duration_scale = 1,
                                    symptomatic_isolation_rate = 0.0,
                                    asymptomatic_contagion_probability = 0.1,
                                    risk_tolerance_mean = 0.7,
                                    risk_tolerance_stdev = 0.2,
                                    herding_behavior_mean = 0.7,
                                    herding_behavior_stdev = 0.2,
                                    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
                                    spreading_rate = normal_ci(2.41, 3.90, 20)
                                ),
                   'listeners': [
                            ["Propaganda", 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_ALL, 1]
                        ],
                        'home_grid_height': 6,
                        'home_grid_width' : 6,
                        'school_height' : 2,
                        'school_width' : 2,
                        'work_height' : 3,
                        'work_width' : 3
                  },
    # 6) No clumpiness, long lockdown (very little spreading).
    'scenario_6': { 'parameters': SimulationParameters(
                                    mask_user_rate = 0.0,
                                    mask_efficacy = 0.0,
                                    imune_rate = 0.01,
                                    initial_infection_rate = 0.01,
                                    hospitalization_capacity = 0.05,
                                    latency_period_shape = 3,
                                    latency_period_scale = 1, 
                                    incubation_period_shape = 6,
                                    incubation_period_scale = 1, 
                                    mild_period_duration_shape = 14,
                                    mild_period_duration_scale = 1,
                                    hospitalization_period_duration_shape = 12,
                                    hospitalization_period_duration_scale = 1,
                                    symptomatic_isolation_rate = 0.0,
                                    asymptomatic_contagion_probability = 0.1,
                                    risk_tolerance_mean = 0.7,
                                    risk_tolerance_stdev = 0.2,
                                    herding_behavior_mean = 0.7,
                                    herding_behavior_stdev = 0.2,
                                    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
                                    spreading_rate = normal_ci(2.41, 3.90, 20)
                                ),
                   'listeners': [
                            ["AddPolicy", SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_RETAIL, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_FACTORY, 1],
                            ["AddPolicy", SocialPolicy.LOCKDOWN_OFFICE, 1],
                        ],
                        'home_grid_height': 6,
                        'home_grid_width' : 6,
                        'school_height' : 6,
                        'school_width' : 6,
                        'work_height' : 6,
                        'work_width' : 6
                  },
    
}

**To pick one of them**, simply change the scenario variable below:

In [5]:
# Selecting scenario
scenario = 'scenario_1'
parameters = scenarios_dict[scenario]['parameters']
listeners = scenarios_dict[scenario]['listeners']
home_grid_height = scenarios_dict[scenario]['home_grid_height']
home_grid_width = scenarios_dict[scenario]['home_grid_width']
school_height = scenarios_dict[scenario]['school_height']
school_width = scenarios_dict[scenario]['school_width']
work_height = scenarios_dict[scenario]['work_height']
work_width = scenarios_dict[scenario]['work_width']

## Running the server

In [6]:
# Setting parameters and adding listeners
set_parameters(parameters)
model = CovidModel()
l = []
ls= copy.deepcopy(listeners)
for listener in ls:
    print(listener)
    funct = listener.pop(0)
    listener[:0]=[model]
    l.append(globals()[funct](*listener))
for k in l:
    model.add_listener(k)

# Setting up grid layout
setup_grid_layout(model, population_size, home_grid_height, home_grid_width, work_height, work_width, school_height, school_width)
statistics = BasicStatistics(model)
model.add_listener(statistics)

# Running server
text1= InfectedText("COVID 19 Simulation: Grid Visualization")
grid = CanvasGrid(agent_portrayal, home_grid_width, home_grid_height, 600, 600)
text2= InfectedText("Agent State Evolution Chart")
chart = ChartModule([{"Label": "Infected",
                      "Color": "Red"},
                    {"Label": "Hospitalized",
                      "Color": "Orange"},
                    {"Label": "Dead",
                      "Color": "Black"}],
                    data_collector_name='datacollector',
                    canvas_width=300
                   )

server = ModularServer(model, CovidModel,
                       [text1, grid, text2, chart],
                       "Covid Model",
                       {"N":100, "width":10, "height":10})
server.port = 8521 # The default
server.launch()

Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running